## Prepare the dataset

In [1]:
import os
import nltk
import json
# from nltk import sent_tokenize
from tqdm import tqdm

In [2]:
z

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
# path = "./data/"
# upper_folder_list = os.listdir(path)

# total_sentence_list = []
# for upper_dir_name in upper_folder_list:
#     file_list = os.listdir(os.path.join(path, upper_dir_name))
#     for dir_name in file_list:
#         file_path = os.path.join(path, upper_dir_name, dir_name)
#         json_file_list = os.listdir(file_path)
        
#         for json_file_name in tqdm(json_file_list):
#             json_file_path = os.path.join(file_path, json_file_name)
            
#             with open(json_file_path, 'r', encoding='UTF-8') as f:
#                 json_object = json.load(f)

#             for line in json_object['SJML']['text']:
#                 raw_text = line['content'].replace('..', '.')
#                 raw_text_list = sent_tokenize(raw_text)
                
#                 for item in raw_text_list:
#                     if len(item) < 5:
#                         continue
#                     else:
#                         total_sentence_list.append(item.strip())
#     break
# # import json

# # with open('input.json') as f:
# #     json_object = json.load(f)

In [3]:
import pickle

with open('total_sentence_list.pickle', 'rb') as f:
    total_sentence_list = pickle.load(f)

In [4]:
from datasets import Dataset

raw_datasets = Dataset.from_dict({"text":total_sentence_list})

: 

: 

In [ ]:
print(len(raw_datasets))
raw_datasets[10]

53742969


{'text': 'sk텔레콤 계열 mvno 사업자 sk텔링크가 2위를 유지하고 있어, 이번 합병이 성사되면 sk텔레콤 계열사가 알뜰폰 시장 30% 이상을 가져가게 된다.'}

In [ ]:
# from transformers import AutoTokenizer
# import multiprocessing
# # from tqdm import tqdm


# # training a tokenizer from scratch
# def batch_iterator(batch_size=10000):
#     for i in tqdm(range(0, len(raw_datasets), batch_size)):
#         yield raw_datasets[i:i+batch_size]["text"]
        
# old_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
# tokenizer = old_tokenizer.train_new_from_iterator(text_iterator=batch_iterator(), vocab_size=15000)
# tokenizer.save_pretrained("tokenizer_aihub_news")

In [ ]:
# import pickle
# with open('total_sentence_list.pickle', 'wb') as fw:
#     pickle.dump(total_sentence_list, fw)

In [ ]:
from transformers import AutoTokenizer
import multiprocessing

tokenizer = AutoTokenizer.from_pretrained("tokenizer_aihub_news")
num_proc = multiprocessing.cpu_count()

def group_texts(examples):
    tokenized_inputs = tokenizer(
       examples["text"], return_special_tokens_mask=True, truncation=True, max_length=tokenizer.model_max_length
    )
    return tokenized_inputs

# preprocess dataset
tokenized_datasets = raw_datasets.map(group_texts, batched=True, remove_columns=["text"], num_proc=num_proc)
tokenized_datasets.features

Map (num_proc=16):   0%|          | 0/53742969 [00:00<?, ? examples/s]

: 

: 

In [ ]:
import pickle
with open('tokenized_datasets.pickle', 'wb') as fw:
    pickle.dump(tokenized_datasets, fw)

In [ ]:
from itertools import chain

# Main data processing function that will concatenate all texts from our dataset and generate chunks of
# max_seq_length.
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= tokenizer.model_max_length:
        total_length = (total_length // tokenizer.model_max_length) * tokenizer.model_max_length
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + tokenizer.model_max_length] for i in range(0, total_length, tokenizer.model_max_length)]
        for k, t in concatenated_examples.items()
    }
    return result

tokenized_datasets = tokenized_datasets.map(group_texts, batched=True, num_proc=16)
# shuffle dataset
tokenized_datasets = tokenized_datasets.shuffle(seed=34)

print(f"the dataset contains in total {len(tokenized_datasets)*tokenizer.model_max_length} tokens")

NameError: name 'tokenized_datasets' is not defined

In [ ]:
from transformers import AutoConfig

model_config = AutoConfig.from_pretrained('bert-base-uncased')

In [ ]:
from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained('bert-base-uncased', config=model_config, cache_dir="./BERT_cache")
model.resize_token_embeddings(len(tokenizer))

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir = "./BERT",
    overwrite_output_dir = True,
    do_train = True,
    do_eval = True,
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 32,
    logging_steps = 50,
    prediction_loss_only = True,
    learning_rate = 5e-5,
    weight_decay = 0,
    adam_epsilon = 1e-8,
    max_grad_norm = 1.0,
    num_train_epochs = 1,
    save_steps = -1
)

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, 
                                                mlm=True, 
                                                mlm_probability=0.15,)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets,
)

In [10]:
trainer.train()

/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
1,12.178000
2,9.012200
3,8.871000
4,10.278800
5,8.616600
6,8.454800
7,8.396900
8,8.270100
9,8.145000
10,8.027700
